In [4]:
# default_exp core

# pico_pi_controller

> A service interface for a Raspberry Pi 2/3 to be monitored & controlled from a CircuitPython capable microcontroller. Use on a RPi 4 requires a solution to enable i2c perpheral support on RPi4.

This module uses `pigpio` to enable a peripheral (slave) device; `pigpiod` must be running.

In [2]:
#hide
#from nbdev.showdoc import *

In [3]:
#export
import time
import atexit
import pigpio

# same arbitrary i2c address as pigpio example
I2C_ADDR=0x13
ACK = 5

# disable for nbdev testing
# cleanup on exit
#@atexit.register
#def goodbye():
#    try:
#        e.cancel()
#        pi.bsc_i2c(0) # Disable BSC peripheral
#        pi.stop()
#        print ("bsc_i2c cleanup completed.")
#    except:
#        print ("bsc_i2c error on cleanup.")
#    print('Exiting via atexit')

#export
class PPController():
    def i2c(id, tick):
        global pi

        s, b, d = pi.bsc_i2c(I2C_ADDR)
        if b:
            if d[0] == 5: # ASCII character 'ENQ' Enquiriry
            
                print ("ENQ recieved")
                s, b, d = pi.bsc_i2c(I2C_ADDR,ACK)
            
#disable below for nbdev, to be placed in 'examples' file
            
#pi = pigpio.pi()

#if not pi.connected:
#    exit()

# setup event handler to listen for heartbeat from MCU
#e = pi.event_callback(pigpio.EVENT_BSC, i2c)                                      

# open the bsc_i2c device as an i2c perpheral
#print ("bsc_i2c started.")
#pi.bsc_i2c(I2C_ADDR)                                  

# since using an event handler, sleep forever
#while True:
#    time.sleep(600)

# leftover for CI testing (CI would fail without attached hardware)    
def connect_mcu():
    return True